In [1]:
%matplotlib inline
%reload_ext autoreload

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import seaborn as sns
import logging

FORMAT = '%(asctime)s %(message)s'

logging.basicConfig(level='INFO', format=FORMAT, datefmt='%d-%b-%y %H:%M:%S')
sns.set_context("talk")

In [3]:
from src.load_data import get_sleep_and_prev_run_epochs

sleep_epoch_keys, prev_run_epoch_keys = get_sleep_and_prev_run_epochs()

In [4]:
from src.load_data import load_data

prev_run_data = load_data(prev_run_epoch_keys[0])

12-Oct-21 06:10:15 Loading position info...
12-Oct-21 06:23:16 Loading multiunit...
12-Oct-21 06:25:11 Loading spikes...
12-Oct-21 06:26:17 Finding ripple times...


In [5]:
from src.load_data import load_data

sleep_data = load_data(sleep_epoch_keys[0])

12-Oct-21 06:26:55 Loading position info...


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [6]:
%debug

> /stelmo/edeno/pose_analysis/src/load_data.py(345)_get_pos_dataframe()
    343     struct = get_data_structure(
    344         animals[animal], day, 'posdlc', 'posdlc')[epoch - 1]
--> 345     position_data = struct['data'][0, 0]
    346     field_names = struct['fields'][0, 0][0].split()
    347     time = pd.TimedeltaIndex(



ipdb>  struct['data']


*** IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices


ipdb>  struct


array([], shape=(1, 0), dtype=float64)


ipdb>  q


In [ ]:
from src.parameters import (ANIMALS, PROCESSED_DATA_DIR, WTRACK_EDGE_ORDER,
                            WTRACK_EDGE_SPACING)

continuous_transition_types = [['random_walk_direction2', 'random_walk',            'uniform', 'random_walk',            'random_walk',            'uniform'],  # noqa
                               ['random_walk',            'random_walk_direction1', 'uniform', 'random_walk',            'random_walk',            'uniform'],  # noqa
                               ['uniform',                'uniform',                'uniform', 'uniform',                'uniform',                'uniform'],  # noqa
                               ['random_walk',            'random_walk',            'uniform', 'random_walk_direction1', 'random_walk',            'uniform'],  # noqa
                               ['random_walk',            'random_walk',            'uniform', 'random_walk',            'random_walk_direction2', 'uniform'],  # noqa
                               ['uniform',                'uniform',                'uniform', 'uniform',                'uniform',                'uniform'],  # noqa
                               ]
encoding_group_to_state = ['Inbound', 'Inbound', 'Inbound',
                           'Outbound', 'Outbound', 'Outbound']

clusterless_algorithm = 'multiunit_likelihood_gpu'
clusterless_algorithm_params = {
    'mark_std': 20.0,
    'position_std': 8.0,
}

classifier_parameters = {
    'movement_var': 6.0,
    'replay_speed': 1,
    'place_bin_size': 2.0,
    'continuous_transition_types': continuous_transition_types,
    'discrete_transition_diag': 0.968,
    'clusterless_algorithm': clusterless_algorithm,
    'clusterless_algorithm_params': clusterless_algorithm_params
}

state_names = [
    'Inbound-Forward', 'Inbound-Reverse', 'Inbound-Fragmented',
    'Outbound-Forward', 'Outbound-Reverse', 'Outbound-Fragmented']

In [ ]:
classifier.fit(
    position=prev_run_data["position_info"].linear_position,
    multiunits=prev_run_data["multiunits"]
    is_training=pd.notnull(prev_run_data["position_info"].task),
    track_graph=prev_run_data["track_graph"],
    edge_order=WTRACK_EDGE_ORDER,
    edge_spacing=WTRACK_EDGE_SPACING,
    encoding_group_labels=np.asarray(
        prev_run_data["position_info"].task).astype(str),
    encoding_group_to_state=encoding_group_to_state
)

In [ ]:
ripple_number = 1

start_time = sleep_data['ripple_times'].loc[ripple_number].start_time
end_time = sleep_data['ripple_times'].loc[ripple_number].end_time

results = classifier.predict(
    multiunits=sleep_data['multiunits'].sel(time=slice(start_time, end_time)),
    time=sleep_data['multiunits'].sel(time=slice(start_time, end_time)).time / np.timedelta64(1, 's'),
    state_names=state_names
    )